Copyright **`(c)`** 2021 Giovanni Squillero `<squillero@polito.it>`  
`https://github.com/squillero/computational-intelligence`  
Free for personal or classroom use; see 'LICENCE.md' for details.

In [2]:
import logging
from collections import deque
import numpy as np
from tqdm.notebook import tqdm, trange
from matplotlib import pyplot as plt

logging.basicConfig(format='[%(asctime)s] %(levelname)s: %(message)s', datefmt='%H:%M:%S', level=logging.INFO)

# Set Covering

See [https://en.wikipedia.org/wiki/Set_cover_problem](https://en.wikipedia.org/wiki/Set_cover_problem)

In [3]:
SEED = 42
np.random.seed(SEED)

In [4]:
NUM_SETS = 200
NUM_AREAS = 20
SETS = np.random.random((NUM_SETS, NUM_AREAS)) < .5
COSTS = np.random.randint(1, 10_000+1, size=(NUM_SETS))

def solution_cost(solution):
    return np.sum(COSTS[solution])

def is_valid(solution):
    return np.all(np.any(SETS[solution], axis=0))

def sol2set(solution):
    return frozenset(n for n in range(NUM_SETS) if solution[n])

def set2sol(set_):
    return np.array([x in set_ for x in range(NUM_SETS)])

In [5]:
def tweak(solution):
    new_solution = solution.copy()
    index = None
    while index is None or np.random.random() < .2:
        index = np.random.randint(0, NUM_SETS)
        new_solution[index] = not new_solution[index]
    return new_solution

In [6]:
logging.getLogger().setLevel(logging.INFO)
NUM_SAMPLES = 10
NUM_STEPS = 1000

## Vanilla Steepest Ascent Hill Climber with Replacement

In [7]:
np.random.seed(SEED)

solution = np.random.random((NUM_SETS,)) < .5
best_solution = None
last_improvement = 0

tot_evaluations = 0
for step in tqdm(range(NUM_STEPS)):
    bunch = [tweak(solution) for _ in range(NUM_SAMPLES)]
    candidates = [sol for sol in bunch if is_valid(sol)]
    if not candidates:
        continue
    tot_evaluations += len(candidates)
    costs = [solution_cost(sol) for sol in candidates]

    i = costs.index(min(costs))
    champion = candidates[i]

    if not is_valid(solution) or solution_cost(champion) < solution_cost(solution):
        solution = champion
        if best_solution is None or solution_cost(solution) < solution_cost(best_solution):
            last_improvement = step
            best_solution = np.copy(solution)
            logging.debug(f"Solution: {sol2set(best_solution)} with cost of {solution_cost(best_solution):,}, found in {last_improvement:,} steps")

logging.info(f"Best solution cost: {solution_cost(best_solution):,}, found in {last_improvement:,} steps ({tot_evaluations:,} total evaluations)")

  0%|          | 0/1000 [00:00<?, ?it/s]

[18:05:42] INFO: Best solution cost: 5,891, found in 117 steps (9,769 total evaluations)


## Tabu Search

In [8]:
np.random.seed(SEED)

solution = np.random.random((NUM_SETS,)) < .5
best_solution = None
last_improvement = 0
tabu_list = deque()

tot_evaluations = 0
for step in tqdm(range(NUM_STEPS)):
    bunch = [tweak(solution) for _ in range(NUM_SAMPLES)]
    candidates = [sol for sol in bunch if is_valid(sol) and sol2set(sol) not in tabu_list]
    if not candidates:
        continue
    tot_evaluations += len(candidates)
    costs = [solution_cost(sol) for sol in candidates]

    i = costs.index(min(costs))
    champion = candidates[i]
    tabu_list.append(sol2set(champion))

    if not is_valid(solution) or solution_cost(champion) < solution_cost(solution):
        solution = champion
        if best_solution is None or solution_cost(solution) < solution_cost(best_solution):
            last_improvement = step
            best_solution = np.copy(solution)
            logging.debug(f"Solution: {sol2set(best_solution)} with cost of {solution_cost(best_solution):,}, found in {last_improvement:,} steps")
    
    while len(tabu_list) > 500:
        tabu_list.popleft()

logging.info(f"Best solution cost: {solution_cost(best_solution):,}, found in {last_improvement:,} steps ({tot_evaluations:,} total evaluations)")

  0%|          | 0/1000 [00:00<?, ?it/s]

[18:08:40] INFO: Best solution cost: 5,891, found in 117 steps (4,153 total evaluations)


## Iterated Local search

In [13]:
def new_starting_position(global_, last):
    if global_ is None:
        return np.random.random((NUM_SETS,)) < .5
    else:
        return global_

In [14]:
np.random.seed(SEED)

best_solution = None
last_improvement = 0

tot_evaluations = 0

for epoch in tqdm(range(5), position=0, desc='Epoch'):
    solution = new_starting_position(best_solution, None)
    for step in tqdm(range(2*NUM_STEPS), position=1):
        new_solution = tweak(solution)
        if not is_valid(new_solution):
            continue
        tot_evaluations += 1

        if not is_valid(solution) or solution_cost(new_solution) < solution_cost(solution):
            solution = new_solution
            if best_solution is None or solution_cost(solution) < solution_cost(best_solution):
                last_improvement = step
                best_solution = np.copy(solution)
                logging.debug(f"Solution: {sol2set(best_solution)} with cost of {solution_cost(best_solution):,}, found in {last_improvement:,} steps")

logging.info(f"Best solution cost: {solution_cost(best_solution):,}, found in {last_improvement:,} steps ({tot_evaluations:,} total evaluations)")

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

[18:17:24] INFO: Best solution cost: 1,589, found in 1,210 steps (9,847 total evaluations)
